# Introducing the Naive Bayes Classifier

Now we will use annotated data to "learn" a sentiment classifier

In [ ]:
# Loading the dependencies
import pandas as pd

from collections import Counter

from nlpia.data.loaders import get_data 

# The casual tokenizer can handle emoticons, unusual punctuation and slang better than the TreeBank tokenizer
from nltk.tokenize import casual_tokenize

## Setting up the "corpus"

Loading the movies corpus from Hutto movies

In [ ]:
movies = get_data('hutto_movies')

# Looking at some of the first instances
movies.head().round(2)

### Getting a description of the data (look at the range)

In [ ]:
movies.describe().round(2)

In [ ]:
# Helps display wide DataFrames in the console, so they look prettier
pd.set_option('display.width', 75)
movies.sentiment

### Loading the data into a DataFrame through a list of dictionaries

In [ ]:
bags_of_words = []

for text in movies.text:
    bags_of_words.append(Counter(casual_tokenize(text)))

df_bows = pd.DataFrame.from_records(bags_of_words)

# from_records() is a DataFrame constructor.
# INPUT: a sequence (list) of dictionaries
# OUTPUT: a DF with columns for all the keys and associated values. Missing values become NaN
print(df_bows)

In [ ]:
# So we fill them with 0:
df_bows = df_bows.fillna(0).astype(int)
print(df_bows)

### Let us look at the shape

Spoiler: A BoW can explode in size; even more when no normalisation is applied at all


In [ ]:
df_bows.shape



Now, let us see the top instances (it is quite sparse)

In [ ]:
df_bows.head()



**Homework**: Integrate the normalisation pipeline to see the difference


In [ ]:
print(df_bows.head()[list(bags_of_words[0].keys())])

#print(df_bows.head()[list(bags_of_words[1].keys())])

### Build the Naive Bayes classifier

All the data is now ready. Let us build a Multinomial NB.

Multimodal NB is suitable for discrete features (e.g., word counts for text classification). 

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [ ]:
movies.sentiment > 0

Now we can train ("fit") our model

In [ ]:
# We are converting the class from float to Boolean, 
# as this classifier only supports discrete labels 
nb = nb.fit(df_bows, movies.sentiment > 0)

### We have a model and we can predict!

In [ ]:
# predict_proba() gets continious-value predictions.
# We multiply and subtract it to convert the output to range [-4,4]

#print(predictions[:10])
# TODO there seems to be an error in th ebook code. 
# predict_proba returns the scores for all the classes (2) and we aim at
# assigning only the one for the positive class. 
# I had to to the following trick instead of the original
# movies['predicted_sentiment'] = nb.predict(df_bows) * 8 - 4
predictions = nb.predict_proba(df_bows) * 8 - 4 
movies['predicted_sentiment'] = [x[1] for x in predictions]

# FROM HERE

In [ ]:
[-4,4]

probability [0,1]

0* 8 -> 0
1 * 8 -> 8

[0,8] -4 -> [-4,4]

Now, we compute the Mean Absolut Error (MAE) "a measure of difference between two continuous variables"

In [ ]:

movies['error'] = (movies.predicted_sentiment - movies.sentiment).abs()
# This is the mean absolute error (MAE)
movies.error.mean().round(1)


Now, let us see some gold and predicted sentiments, together with the binary classification

In [ ]:
# Gold standard is positive
movies['sentiment_ispositive'] = (movies.sentiment > 0).astype(int)

# Prediction is positive
movies['predicted_ispositive'] = (movies.predicted_sentiment > 0).astype(int)

# Let us have an overview of gold standard vs prediction
movies['''sentiment predicted_sentiment sentiment_ispositive predicted_ispositive'''.split()].head(8)

In [ ]:
# And this is the percentage of "thumbs up" rating correctly predicted    
(movies.predicted_ispositive == movies.sentiment_ispositive).sum() / len(movies)
